# Demonstration of PET OSEM reconstruction with SIRF
This demonstration shows how to use OSEM as implemented in SIRF. It also suggests some exercises for reconstruction with and without attenuation etc.

The notebook is currently set-up to use prepared data with a single slice of an XCAT phantom, with a low resolution scanner, such that all results can be obtained easily on a laptop. Of course, the code will work exactly the same for any sized data.

Authors: Kris Thielemans and Evgueni Ovtchinnikov  
First version: June 2021

CCP SyneRBI Synergistic Image Reconstruction Framework (SIRF).  
Copyright 2015 - 2018, 2021 University College London.

This is software developed for the Collaborative Computational Project in Synergistic Reconstruction for Biomedical Imaging (http://www.ccpsynerbi.ac.uk/).

SPDX-License-Identifier: Apache-2.0

# What is OSEM?
The following is just a very brief explanation of the concepts behind OSEM.

PET reconstruction is commonly based on the *Maximum Likelihood Estimation (MLE)* principle. The *likelihood* is the probability to observe some measured data given a (known) image. MLE attempts to find the image that maximises this likelihood. This needs to be done iteratively as the system of equations is very non-linear.

A common iterative method uses *Expectation Maximisation*, which we will not explain here. The resulting algorithm is called *MLEM* (or sometimes *EMML*). However, it is rather slow. The most popular method to increase computation speed is to compute every image update based on only a subset of the data. Subsets are nearly always chosen in terms of the "views" (or azimuthal angles). The *Ordered Subsets Expectation Maximisation (OSEM)* cycles through the subsets. More on this in another notebook, but here we just show how to use the SIRF implementation of OSEM.

OSEM is (still) the most common algorithm in use in clinical PET.

# Initial set-up

### make sure figures appears inline and animations works

In [ ]:
%matplotlib widget


### Imports and setting-up of the data and working directory for the notebook.

Please make sure that you have run the `download_data.sh` script first. See the [Introductory/introduction notebook](../Introductory/introduction.ipynb) for more information.

In [ ]:
#%% Initial imports etc
import notebook_setup

import numpy
from numpy.linalg import norm
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import os
import sys
import shutil
#import scipy
import sirf.STIR as pet
from sirf.Utilities import examples_data_path
from sirf_exercises import exercises_data_path

# define the directory with input files for this notebook
data_path = os.path.join(examples_data_path('PET'), 'thorax_single_slice')

### redirect STIR messages to some files
STIR output can be a bit verbose. There are currently also some spurious warnings. To avoid distracting you, we redirect these messages to some files. You can check these if things go wrong.


In [ ]:
msg_red = pet.MessageRedirector('info.txt', 'warnings.txt')

### our usual handy function definitions

In [ ]:
def plot_2d_image(idx,vol,title,clims=None,cmap="viridis"):
    """Customized version of subplot to plot 2D image"""
    plt.subplot(*idx)
    plt.imshow(vol,cmap=cmap)
    if not clims is None:
        plt.clim(clims)
    plt.colorbar(shrink=.6)
    plt.title(title)
    plt.axis("off")

## We will first create some simulated data from ground-truth images
see previous notebooks for more information.

In [ ]:
#%% Read in images
image = pet.ImageData(os.path.join(data_path, 'emission.hv'))
attn_image = pet.ImageData(os.path.join(data_path, 'attenuation.hv'))

In [ ]:
#%% display
im_slice = image.dimensions()[0]//2
plt.figure(figsize=(9, 4))
plot_2d_image([1,2,1],image.as_array()[im_slice,:,:,], 'emission image')
plot_2d_image([1,2,2],attn_image.as_array()[im_slice,:,:,], 'attenuation image')
plt.tight_layout()

In [ ]:
#%% save max for future displays
cmax = image.max()*.6

In [ ]:
#%% create acquisition model
acq_model = pet.AcquisitionModelUsingRayTracingMatrix()
template = pet.AcquisitionData(os.path.join(data_path, 'template_sinogram.hs'))
acq_model.set_up(template, image)

In [ ]:
#%% simulate data using forward projection
acquired_data=acq_model.forward(image)

In [ ]:
#%% Display bitmaps of a middle sinogram
acquired_data.show(im_slice,title='Forward projection')

# Reconstruction via a SIRF reconstruction class
While you can write your own reconstruction algorithm by using `AcquisitionModel` etc (see  other notebooks), SIRF provides a few reconstruction clases. We show how to use the OSEM implementation here.

## step 1: create the objective function

In PET, the iterative algorithms in SIRF rely on an objective function (i.e. the function to maximise).
In PET, this is normally the Poisson log-likelihood. (We will see later about adding prior information).

In [ ]:
obj_fun = pet.make_Poisson_loglikelihood(acquired_data)

We could set acquisition model but the default (ray-tracing) is in this case ok. See below for more information. You could do this as follows.
```
obj_fun.set_acquisition_model(acq_model)
```

We could also add a prior, but we will not do that here (although the rest of the exercise would still work). 

## step 2: create OSMAPOSL reconstructor
The `sirf.STIR.OSMAPOSLReconstructor` class implements the *Ordered Subsets Maximum A-Posteriori One Step Late algorithm*. That's quite a mouthful! We will get round to the "OSL" part, which is used to incorporate prior information. However, without a prior, this algorithm is identical to *Ordered Subsets Expectation Maximisation* (OSEM).

In [ ]:
recon = pet.OSMAPOSLReconstructor()
recon.set_objective_function(obj_fun)
# use 4 subset and 60 image updates. This is not too far from clinical practice.
recon.set_num_subsets(4)
num_subiters=60
recon.set_num_subiterations(num_subiters)

## step 3: use this reconstructor!

We first create an initial image. Passing this image automatically gives the dimensions of the output image.
It is common practice to initialise OSEM with a uniform image. Here we use a value which is roughly of the correct scale, although this value doesn't matter too much (see discussion in the OSEM_DIY notebook).

Then we need to set-up the reconstructor. That will do various checks and initial computations.

And then finally we call the `reconstruct` method.

In [ ]:
#initialisation
initial_image=image.get_uniform_copy(cmax / 4)
recon.set_current_estimate(initial_image)
# set up the reconstructor
recon.set_up(initial_image)
# do actual recon
recon.process()
reconstructed_image=recon.get_output()

display of images

In [ ]:
plt.figure(figsize=(9, 4))
plot_2d_image([1,2,1],image.as_array()[im_slice,:,:,],'ground truth image',[0,cmax*1.2])
plot_2d_image([1,2,2],reconstructed_image.as_array()[im_slice,:,:,],'reconstructed image',[0,cmax*1.2])
plt.tight_layout();

## step 4: write to file
You can ask the `OSMAPOSLReconstructor` to write images to file every few sub-iterations, but this is by default disabled. We can however write the image to file from SIRF.

For each "engine" its default file format is used, which for STIR is Interfile.

In [ ]:
reconstructed_image.write('OSEM_result.hv')

You can also use the `write_par` member to specify a STIR parameter file to write in a different file format, but this is out of scope for this exercise.

# Including a more realistic acquisition model
The above steps were appropriate for an acquisition without attenuation etc. This is of course not appropriate for measured data.

Let us use some things we've learned from the [image_creation_and_simulation notebook](image_creation_and_simulation.ipynb). First thing is to create a new acquisition model, then we need to use it to simulate new data, and finally to use it for the reconstruction.

In [ ]:
# create attenuation
acq_model_for_attn = pet.AcquisitionModelUsingRayTracingMatrix()
asm_attn = pet.AcquisitionSensitivityModel(attn_image, acq_model_for_attn)
asm_attn.set_up(template)
attn_factors = asm_attn.forward(template.get_uniform_copy(1))
asm_attn = pet.AcquisitionSensitivityModel(attn_factors)

In [ ]:
# create acquisition model
acq_model = pet.AcquisitionModelUsingRayTracingMatrix()
# we will increase the number of rays used for every Line-of-Response (LOR) as an example
# (it is not required for the exercise of course)
acq_model.set_num_tangential_LORs(5)
acq_model.set_acquisition_sensitivity(asm_attn)
# set-up
acq_model.set_up(template,image)

In [ ]:
# simulate data
acquired_data = acq_model.forward(image)

In [ ]:
# let's add a background term of a reasonable scale
background_term = acquired_data.get_uniform_copy(acquired_data.max()/10)
acq_model.set_background_term(background_term)
acquired_data = acq_model.forward(image)

In [ ]:
# create reconstructor
obj_fun = pet.make_Poisson_loglikelihood(acquired_data)
obj_fun.set_acquisition_model(acq_model)
recon = pet.OSMAPOSLReconstructor()
recon.set_objective_function(obj_fun)
recon.set_num_subsets(4)
recon.set_num_subiterations(60)

In [ ]:
# initialisation and reconstruction
recon.set_current_estimate(initial_image)
recon.set_up(initial_image)
recon.process()
reconstructed_image=recon.get_output()

In [ ]:
# display
plt.figure(figsize=(9, 4))
plot_2d_image([1,2,1],image.as_array()[im_slice,:,:,],'ground truth image',[0,cmax*1.2])
plot_2d_image([1,2,2],reconstructed_image.as_array()[im_slice,:,:,],'reconstructed image',[0,cmax*1.2])
plt.tight_layout();

# Exercise: write a function to do an OSEM reconstruction
The above lines still are quite verbose. So, your task is now to create a function that includes these steps, such that you can avoid writing all those lines all over again.

For this, you need to know a bit about Python, but mostly you can copy-paste lines from above.

Let's make a function that creates an acquisition model, given some input. Then we can write OSEM function that does the reconstruction.

Below is a skeleton implementation. Look at the code above to fill in the details.

To debug your code, it might be helpful at any messages that STIR writes. By default these are written to the terminal, but this is not helpful when running in a jupyter notebook. The line below will redirect all messages to files which you can open via the `File>Open` menu.

In [ ]:
msg_red = pet.MessageRedirector('info.txt', 'warnings.txt', 'errors.txt')

Note that they will be located in the current directory.

In [ ]:
%pwd

In [ ]:
def create_acq_model(attn_image, background_term):
    '''create a PET acquisition model.
    
    Arguments:
    attn_image: the mu-map
    background_term: bakcground-term as an sirf.STIR.AcquisitionData
    '''
    # acq_model_for_attn = ...
    # asm_model = ...
    acq_model = pet.AcquisitionModelUsingRayTracingMatrix();
    # acq_model.set_...
    return acq_model

In [ ]:
def OSEM(acq_data, acq_model, initial_image, num_subiterations, num_subsets=1):
    '''run OSEM
    
    Arguments:
    acq_data: the (measured) data
    acq_model: the acquisition model
    initial_image: used for initialisation (and sets voxel-sizes etc)
    num_subiterations: number of sub-iterations (or image updates)
    num_subsets: number of subsets (defaults to 1, i.e. MLEM)
    '''
    #obj_fun = ...
    #obj_fun.set...
    recon = pet.OSMAPOSLReconstructor()
    #recon.set_objective_function(...)
    #recon.set...
    recon.set_current_estimate(initial_image)
    recon.set_up(initial_image)
    recon.process()
    return recon.get_output()

Now test it with the above data.

In [ ]:
acq_model = create_acq_model(attn_image, background_term)
my_reconstructed_image = OSEM(acquired_data, acq_model, image.get_uniform_copy(cmax), 30, 4)

# Exercise: reconstruct with and without attenuation
In some cases, it can be useful to reconstruct the emission data without taking attenuation (or even background terms) into account. One common example is to align the attenuation image to the emission image.

It is easy to do such an *no--attenuation-correction (NAC)* recontruction in SIRF. You need to create an `AcquisitionModel` that does not include the attenuation factors, and use that for the reconstruction. (Of course, in a simulation context, you would still use the full model to do the simulation).

Implement that here and reconstruct the data with and without attenuation to see visually what the difference is in the reconstructed images. If you have completed the previous exercise, you can use your own functions to do this.

Hint: the easiest way would be to take the existing attenuation image, and use `get_uniform_copy(0)` to create an image where all $\mu$-values are 0. Another (and more efficient) way would be to avoid creating the `AcquisitionSensitivityModel` at all.

# Final remarks

In these exercises we have used attenuation images of the same size as the emission image. This is easier to code and for display etc, but it is not a requirement of SIRF.

In addition, we have simulated and reconstructed the data with the same `AcquisitionModel` (and preserved image sizes). This is also convenient, but not a requirement (as you've seen in the NAC exercise). In fact, do not write your next paper using this "inverse crime". The problem is too 